# Data exploration

We are exploring the data provided by Canada census.

Source of Data:
https://www12-2021.statcan.gc.ca/census-recensement/2021/dp-pd/prof/details/download-telecharger.cfm?Lang=E
https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/index2021-eng.cfm?year=21

In [1]:
import numpy as np
import pandas as pd
import os,glob


In [2]:

# Load the data
csv = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/data/98-401-X2021006_English_CSV_data_Territories.csv'
df = pd.read_csv(csv,encoding="ISO-8859-1")
characteristic_categories = pd.read_csv("characteristic_names.csv", encoding="ISO-8859-1")
categories = characteristic_categories["Category"].unique()
print(df.shape)

(475741, 23)


In [2]:
# Dropping columns that are not necessary
df = df.drop(["CHARACTERISTIC_NOTE", "SYMBOL", "SYMBOL.1", "SYMBOL.2", "SYMBOL.3", "SYMBOL.4", "SYMBOL.5", "C10_RATE_TOTAL", "C11_RATE_MEN+", "C12_RATE_WOMEN+"], axis=1)
df.fillna(0)

,CENSUS_YEAR,DGUID,ALT_GEO_CODE,GEO_LEVEL,GEO_NAME,TNR_SF,TNR_LF,DATA_QUALITY_FLAG,CHARACTERISTIC_ID,CHARACTERISTIC_NAME,C1_COUNT_TOTAL,C2_COUNT_MEN+,C3_COUNT_WOMEN+
0,2021,2021A000011124,1,Country,Canada,3.1,4.3,20000,1,"Population, 2021",36991981.0,0.0,0.0
1,2021,2021A000011124,1,Country,Canada,3.1,4.3,20000,2,"Population, 2016",35151728.0,0.0,0.0
2,2021,2021A000011124,1,Country,Canada,3.1,4.3,20000,3,"Population percentage change, 2016 to 2021",5.2,0.0,0.0
3,2021,2021A000011124,1,Country,Canada,3.1,4.3,20000,4,Total private dwellings,16284235.0,0.0,0.0
4,2021,2021A000011124,1,Country,Canada,3.1,4.3,20000,5,Private dwellings occupied by usual residents,14978941.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475736,2021,2021S051262080015,62080015,Dissemination area,62080015,0.0,0.0,9999,1383,Uralic languages,0.0,0.0,0.0
475737,2021,2021S051262080015,62080015,Dissemination area,62080015,0.0,0.0,9999,1384,Estonian,0.0,0.0,0.0
475738,2021,2021S051262080015,62080015,Dissemination area,62080015,0.0,0.0,9999,1385,Finnish,0.0,0.0,0.0
475739,2021,2021S051262080015,62080015,Dissemination area,62080015,0.0,0.0,9999,1386,Hungarian,0.0,0.0,0.0


In [3]:
# Isolating for just Canada to try for one section
canada_df = df[(df["GEO_LEVEL"] != "Country")  ] 

# size of canada_df
canada_df.shape

(474354, 13)

In [4]:
# Creating a database and merging the characteristic ids for categories we want to keep
new_df = pd.merge(canada_df, characteristic_categories, on="CHARACTERISTIC_ID", how="left")
new_df = new_df[new_df['Keep or Not Keep'] == "Keep"]
new_df = new_df[new_df['Category'] != "Remove"]

In [5]:
# Creating a different category for each database
category_df = {}
for category in categories:
    category_df[category] = new_df[new_df["Category"] == category]
category_df.pop("summary",None)
category_df.pop("remove",None)


,CENSUS_YEAR,DGUID,ALT_GEO_CODE,GEO_LEVEL,GEO_NAME,TNR_SF,TNR_LF,DATA_QUALITY_FLAG,CHARACTERISTIC_ID,CHARACTERISTIC_NAME_x,C1_COUNT_TOTAL,C2_COUNT_MEN+,C3_COUNT_WOMEN+,CHARACTERISTIC_NAME_y,Depth,Keep or Not Keep,Category,Flag,NEW_CHARACTERISTIC_NAME
125,2021,2021A000260,60,Territory,Yukon,4.3,10.5,10,126,Total - Income statistics in 2020 for the popu...,32775.0,16185.0,16595.0,Total - Income statistics in 2020 for the popu...,0,Keep,remove,Category,Total - Income statistics in 2020 for the popu...
127,2021,2021A000260,60,Territory,Yukon,4.3,10.5,10,128,Average total income in 2020 among recipie...,64200.0,67800.0,60750.0,Average total income in 2020 among recipie...,4,Keep,remove,Category,Average total income in 2020 among recipients ($)
129,2021,2021A000260,60,Territory,Yukon,4.3,10.5,10,130,Average after-tax income in 2020 among rec...,54750.0,56900.0,52700.0,Average after-tax income in 2020 among rec...,4,Keep,remove,Category,Average after-tax income in 2020 among recipie...
131,2021,2021A000260,60,Territory,Yukon,4.3,10.5,10,132,Average market income in 2020 among recipi...,60700.0,65500.0,56050.0,Average market income in 2020 among recipi...,4,Keep,remove,Category,Average market income in 2020 among recipients...
133,2021,2021A000260,60,Territory,Yukon,4.3,10.5,10,134,Average employment income in 2020 among re...,58050.0,62150.0,54100.0,Average employment income in 2020 among re...,4,Keep,remove,Category,Average employment income in 2020 among recipi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474348,2021,2021S051262080015,62080015,Dissemination area,62080015,0.0,0.0,9999,1382,"Turkic languages, n.i.e.",NaN,NaN,NaN,"Turkic languages, n.i.e.",8,Keep,remove,Category,"Turkic languages, n.i.e."
474350,2021,2021S051262080015,62080015,Dissemination area,62080015,0.0,0.0,9999,1384,Estonian,NaN,NaN,NaN,Estonian,8,Keep,remove,Category,Estonian
474351,2021,2021S051262080015,62080015,Dissemination area,62080015,0.0,0.0,9999,1385,Finnish,NaN,NaN,NaN,Finnish,8,Keep,remove,Category,Finnish
474352,2021,2021S051262080015,62080015,Dissemination area,62080015,0.0,0.0,9999,1386,Hungarian,NaN,NaN,NaN,Hungarian,8,Keep,remove,Category,Hungarian


In [6]:
#removing categories so we just have the different categories
filtered_category = [category for category in categories if category not in ["summary", "remove"]]



In [7]:

# Check if error occured - if error, it will print something out.
for category in filtered_category:
    new_dict = {'category': category}
    new_df = category_df[category]
    total_df = new_df[new_df["Flag"] == "Total"]
    categories_df = new_df[new_df["Flag"] == "Category"]
    
    if ((total_df["C1_COUNT_TOTAL"].sum() - categories_df["C1_COUNT_TOTAL"].sum()) / categories_df["C1_COUNT_TOTAL"].sum() > 0.01):
        print("ERROR OCCURED")
        break


/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_69276/1212918017.py:8: RuntimeWarning: invalid value encountered in double_scalars
  if ((total_df["C1_COUNT_TOTAL"].sum() - categories_df["C1_COUNT_TOTAL"].sum()) / categories_df["C1_COUNT_TOTAL"].sum() > 0.01):


In [8]:
# For each category in filtered_category, create a new csv file with the characteristics we want to keep
for category in filtered_category:
    df_to_csv = category_df[category]
    PATH = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/cleaned-data/' + category
    if not os.path.exists(PATH):
        os.makedirs(PATH)
    csv_category = df_to_csv[df_to_csv["Flag"] == "Category"]
    csv_category.to_csv("./cleaned-data/" + category + "/" + category + ".csv", index=False)

    

In [11]:
results = []
characteristic_categories = pd.read_csv("characteristic_names.csv", encoding="ISO-8859-1")
categories = characteristic_categories["Category"].unique()
filtered_category = [category for category in categories if category not in ["summary", "remove"]]

data_path = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/data/'
for csv_file in glob.glob(os.path.join(data_path, '*.csv')):
    area = csv_file[(len(data_path)+33):-4] 
    df = pd.read_csv(csv_file,encoding="ISO-8859-1")
    df = df.drop(["CHARACTERISTIC_NOTE", "SYMBOL", "SYMBOL.1", "SYMBOL.2", "SYMBOL.3", "SYMBOL.4", "SYMBOL.5", "C10_RATE_TOTAL", "C11_RATE_MEN+", "C12_RATE_WOMEN+"], axis=1)
    df.fillna(0)

    canada_df = df[(df["GEO_LEVEL"] != "Country")  ] 

    new_df = pd.merge(canada_df, characteristic_categories, on="CHARACTERISTIC_ID", how="left")
    new_df = new_df[new_df['Keep or Not Keep'] == "Keep"]
    new_df = new_df[new_df['Category'] != "Remove"]

    category_df = {}
    for category in categories:
        category_df[category] = new_df[new_df["Category"] == category]
    category_df.pop("summary",None)
    category_df.pop("remove",None)

    for category in filtered_category:
        new_dict = {'category': category}
        new_df = category_df[category]
        total_df = new_df[new_df["Flag"] == "Total"]
        categories_df = new_df[new_df["Flag"] == "Category"]
        
        if ((total_df["C1_COUNT_TOTAL"].sum() - categories_df["C1_COUNT_TOTAL"].sum()) / categories_df["C1_COUNT_TOTAL"].sum() > 0.01):
            print("ERROR OCCURED")
            break
    for category in filtered_category:
        df_to_csv = category_df[category]
        PATH = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/cleaned-data/' + category
        if not os.path.exists(PATH):
            os.makedirs(PATH)
        csv_category = df_to_csv[df_to_csv["Flag"] == "Category"]
        csv_category.to_csv("./cleaned-data/" + category + "/" + category + "_"+ area + ".csv")
    



/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_69276/505088802.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file,encoding="ISO-8859-1")
/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_69276/505088802.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file,encoding="ISO-8859-1")
/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_69276/505088802.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file,encoding="ISO-8859-1")
/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_69276/505088802.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file,encoding="ISO-8859-1")
/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_69276/5050888

In [8]:
results = []
characteristic_categories = pd.read_csv("characteristic_names.csv", encoding="ISO-8859-1")
categories = characteristic_categories["Category"].unique()
filtered_category = [category for category in categories if category not in ["summary", "remove"]]

data_path = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/data/'
for csv_file in glob.glob(os.path.join(data_path, '*.csv')):
    area = csv_file[(len(data_path)+33):-4] 
    df = pd.read_csv(csv_file,encoding="ISO-8859-1")
    df = df.drop(["CHARACTERISTIC_NOTE", "SYMBOL", "SYMBOL.1", "SYMBOL.2", "SYMBOL.3", "SYMBOL.4", "SYMBOL.5", "C10_RATE_TOTAL", "C11_RATE_MEN+", "C12_RATE_WOMEN+"], axis=1)
    df.fillna(0)
    print(area)
    canada_df = df[(df["GEO_LEVEL"] != "Country")  ] 

    new_df = pd.merge(canada_df, characteristic_categories, on="CHARACTERISTIC_ID", how="left")
    new_df = new_df[new_df['Keep or Not Keep'] == "Keep"]
    new_df = new_df[new_df['Category'] != "Remove"]

    csv_category = new_df[new_df["Category"] == "summary"]

    
    PATH = '/Users/boristsao/Documents/Projects/canada-vis/data-exploration/cleaned-data/summary'
    if not os.path.exists(PATH):
        os.makedirs(PATH)
    
    csv_category.to_csv("./cleaned-data/summary/summary" + "_"+ area + ".csv")

/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_70678/3182646756.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file,encoding="ISO-8859-1")


Prairies


/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_70678/3182646756.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file,encoding="ISO-8859-1")


Quebec


/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_70678/3182646756.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file,encoding="ISO-8859-1")


Ontario


/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_70678/3182646756.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file,encoding="ISO-8859-1")


Atlantic
Territories


/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_70678/3182646756.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file,encoding="ISO-8859-1")


BritishColumbia
